In [41]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

# 例外処理用のlibraryをimport
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import sys

import pandas as pd
import numpy as np

In [42]:
print(selenium.__version__)

4.15.2


## 都道府県ごとのランキングを取得する場合の関数

In [43]:
# 都道府県ごとのランキングデータフレームを作成する
def Pref_cycle(df_pref):
  for i in range(1, 48):
    # 都道府県ごとのデータ取得
    pref_element = driver.find_element(By.XPATH, '//div[@class="wrap"]/nav/div[@class="left"]/select[@name="state"]')
    # 都道府県のselect要素をselectオブジェクトへ変換
    pref_select = Select(pref_element)
    # オブジェクトからvalue属性のリストを作成する
    prefvalue_list = [pref.get_attribute('value') for pref in pref_select.options]
    # 都道府県の要素をドロップダウンから選択して
    pref_select.select_by_value(prefvalue_list[i])
    # 表示ボタンの要素を取得できるまで最大10秒間待機し、2秒ごとに要素が表示されるか確認
    viewer = WebDriverWait(driver, 10, poll_frequency=2).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
    )
    # 要素（表示ボタン）が表示されるまでスクロールする
    driver.execute_script("arguments[0].scrollIntoView();", viewer)
    viewer.click()
    # ページ全体がロードされるのを最大10秒間待機し、2秒ごとに確認
    WebDriverWait(driver, 10, poll_frequency=2).until(
        lambda d: d.execute_script('return document.readyState') == 'complete'
    )
    # top3の店のみ店名とポイントを取得
    store_name_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a')
    store_point_top3 = driver.find_elements(By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="point"]')
    
    # 取得したtop3の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_top3_list = [store_top3.text for store_top3 in store_name_top3]
    store_point_top3_list = [store_top3.text for store_top3 in store_point_top3]
    href_list_top3 = [ hrefs.get_attribute('href') for hrefs in store_name_top3]
    
    # top4~50の店名とポイントを取得
    store_name = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a')
    store_point = driver.find_elements(By.XPATH, '//table[@class="rank"]/tbody/tr/td[@class="point"]')

    # 取得したtop4~50の店名とポイント、詳細ページのurlをリスト形式で保存
    store_name_list = [store.text for store in store_name]
    store_point_list = [store.text for store in store_point]
    href_list = [hrefs.get_attribute('href') for hrefs in store_name]
    Top3_detail(store_name_top3_list, store_point_top3_list, href_list_top3, df_pref)
    Top4to50_detail(store_name_list, store_point_list, href_list, df_pref)

#絞込に従う、ランキングtop3店の詳細情報を取得する
def Top3_detail(name_list, points_list, href_list, df_pref):
  # top3の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    # リストに保存したurlに遷移する
    safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = get_address()
    # 緯度経度の要素を取得する
    latitude,  longitude = get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = get_holiday()
    mk_detail_list(name_list[i], points_list[i], address, latitude, longitude, parking, holiday, df_pref)
        
        
#絞込に従う、ランキングtop4~5店の詳細情報を取得する
def Top4to50_detail(name_list, points_list, href_list, df_pref):
  # top4~50の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = get_address()
    # 緯度経度の要素を取得する
    latitude,  longitude = get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = get_holiday()
    mk_detail_list(name_list[i], points_list[i], address, latitude, longitude, parking, holiday, df_pref)
  # ランキングのページに戻り、都道府県のプルダウンを取得できるようにする
  driver.get('https://ramendb.supleks.jp/rank')
  # ページ全体がロードされるのを最大10秒間待機する
  WebDriverWait(driver, 10).until(
      lambda d: d.execute_script('return document.readyState') == 'complete'
  )
    
def get_address():
  try:
    address = driver.find_element(By.XPATH, '//tbody/tr/td/span[@itemprop="address"]').text
  # 住所が取得できなければ、Noneを取得
  except :
    address = None
  return address
        
def get_map():
  try:
    map_data = driver.find_element(By.ID, 'gpetitembedmap')
    map_data = map_data.get_attribute('src')
    # srcの要素値を緯度経度だけ切り離す
    # q= という文字が入るので、スライスで取り除く
    map_data = map_data.split('&')[1][2:]
    # 緯度経度に分割する
    map_data = map_data.split(',')
    latitude = map_data[0]
    longitude = map_data[1]
  # 緯度経度が取得できなければ、Noneを取得
  except :
    latitude = None
    longitude = None
  return latitude, longitude
        
def get_parking():
  try:
    th_element = driver.find_element(By.XPATH, '//tbody/tr/th[contains(text(), "駐車場")]')
    parking = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except :
    parking = None
  return parking
        
def get_holiday():
  try:
    th_element = driver.find_element(By.XPATH, '//tbody/tr/th[contains(text(), "定休日")]')
    holiday = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except:
    holiday = None
  return holiday

# 店舗名とポイント、店舗詳細を店舗ごとにリスト化する
def mk_detail_list(name, points, address, latitude, longitude, parking, holiday, df_pref):
  detail_list = [name]
  detail_list.append(points)
  detail_list.append(address)
  detail_list.append(latitude)
  detail_list.append(longitude)
  detail_list.append(parking)
  detail_list.append(holiday)
  
  df_pref.loc[len(df_pref)] = detail_list

### 詳細ページに遷移するための関数
* WebdriverExceptionが発生した場合に待機して、リロードを行う<br>
* webdriverExcdeptionに対する対応（仮）
5回リロードしても同じエラーが発生する場合には強制終了を行う

In [44]:
def safe_navigation(href, i = 0):
    try:
        driver.get(href)
        # ページ全体がロードされるのを最大10秒間待機し、2秒ごとに確認
        WebDriverWait(driver, 10, poll_frequency=2).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )
    except WebDriverException as e:
        if i >=5:
            # 強制的に終了する
            try:
                sys.exit()
            except SystemExit:
                print('ページ遷移不可のため、強制終了')
                raise
        print(f"Navigation error: {e}")
        i += 1
        safe_navigation(href,i)

## 分類ごとのランキングを取得する関数

In [45]:
# 分類ごとのランキング表を作成する
def Kind_cycle(kinds_another_list, pager):
    for i in pager:
        # 分類別ごとのデータ取得
        # 分類別、スープ別のselect要素を取得する
        kinds_sel_ele = driver.find_element(By.XPATH, '//select[@name="style"]')
        # 分類別の要素をオブジェクトへ変換する
        kinds_sel_obj = Select(kinds_sel_ele)
        # オブジェクトの、value属性のリストを作成する
        kinds_val_list = [kind.get_attribute('value') for kind in kinds_sel_obj.options]
        kinds_tex_list = [kind.text for kind in kinds_sel_obj.options]
        # 都道府県の要素をドロップダウンから選択して
        kinds_sel_obj.select_by_value(kinds_val_list[i])
        # 表示ボタンの要素を取得できるまで待機（最大十秒）
        viewer = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@class="left"]/input[@class="formbtn"]'))
        )
        # 要素（表示ボタン）が表示されるまでスクロールする
        driver.execute_script("arguments[0].scrollIntoView();", viewer)
        viewer.click()
        
        # top3の店のみ店名を取得できるまで2秒おきに確認しながら、最大10秒待機
        store_name_top3 = WebDriverWait(driver, 10, poll_frequency=2).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a'))
        )
        # 取得したtop3の店名をリスト形式で保存
        store_name_top3_list = [store_top3.text for store_top3 in store_name_top3]
        
        # top4~50の店名を取得できるまで最大10秒待機
        store_name = WebDriverWait(driver, 10).until(
            EC.visibility_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a'))
        )
        # 取得したtop4~50の店名をリスト形式で保存
        store_name_list = [store.text for store in store_name]
        
        kinds_another_list.append(store_name_top3_list + store_name_list)
    
    # 種別ごとのランキングをカラムにするため、numpy形式に変換して転置する
    kinds_another_nplist = np.array(kinds_another_list).T
    return kinds_another_nplist, kinds_tex_list

## 都道府県ごとのランキングを取得したい場合に実行する

In [46]:
options = Options()

# 必要に応じてoptionsを設定
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(5)

driver.get('https://ramendb.supleks.jp/rank')

# # データフレームの枠組みを作成する
columns_list_detail = ['name', 'point', 'address', 'latitude', 'longitude', 'parking', 'holiday']
df_pref = pd.DataFrame(columns = columns_list_detail)
Pref_cycle(df_pref)
df_pref.to_csv('../data/pref_rank.csv')

## 種類ごとのランキングが取得したい場合に実行する

In [47]:
options = Options()

# 必要に応じてoptionsを設定
driver = webdriver.Chrome(options=options)
# 暗黙的な待機
driver.implicitly_wait(5)

driver.get('https://ramendb.supleks.jp/rank')

# 種別ごとのTOP50の店名のリストを格納するリストを作成
kinds_another_list = []
# 実行するプルダウンのリスト
pager = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16]

kinds_another_nplist, kinds_tex_list = Kind_cycle(kinds_another_list, pager)

kinds_tex_list.remove('全て')
while "その他" in kinds_tex_list:
    kinds_tex_list.remove("その他")

# 取得した種類別のランキングをカラム別にcsv形式に変換
df_kindsrank = pd.DataFrame(kinds_another_nplist, columns = kinds_tex_list)
df_kindsrank.to_csv("../data/kinds_of_rank.csv",index=False)